In [ ]:
import json, pathlib, re, collections, plotly.graph_objects as go
from collections import Counter, defaultdict

In [ ]:
# ----------  palette ----------
CLR_EOL      = "#ff6c6c"
CLR_NON_EOL  = "#6c6cff"
CLR_BG       = "#e9eef6"
CLR_TEXT     = "#2c3e50"
FONT_FAMILY  = "Arial"

# ----------  paths ----------
ROOT   = pathlib.Path(__file__).resolve().parent       # data_nick/
OUTDIR = ROOT / "charts"
OUTDIR.mkdir(exist_ok=True)

PORT_RE = re.compile(r"eol_(\d+)\.json", re.I)
ASN_RE  = re.compile(r"(AS\d+)", re.I)

# ----------  accumulators ----------
totals_per_port   = Counter()
eols_per_port     = Counter()
totals_per_asn    = Counter()
eols_per_asn      = Counter()
product_counter   = Counter()
product_eol       = Counter()
heatmap           = defaultdict(dict)  # asn → port → (eol,total)

In [ ]:
def process_file(f: pathlib.Path):
    m_port = PORT_RE.match(f.name)
    m_asn  = ASN_RE.search(str(f.parent))
    if not (m_port and m_asn):
        return
    port, asn = m_port.group(1), m_asn.group(1)

    data   = json.load(f.open(encoding="utf-8"))
    totals = len(data)
    eols   = sum(1 for row in data if row.get("is_eol"))

    totals_per_port[port] += totals
    eols_per_port[port]   += eols
    totals_per_asn[asn]   += totals
    eols_per_asn[asn]     += eols
    prev_e, prev_t = heatmap[asn].get(port, (0, 0))
    heatmap[asn][port] = (prev_e + eols, prev_t + totals)

    for row in data:
        prod = row.get("server", "unknown").split("/")[0].strip()
        product_counter[prod] += 1
        if row.get("is_eol"):
            product_eol[prod] += 1

for f in ROOT.glob("AS*/eol_*.json"):
    process_file(f)